In [ ]:
from matplotlib import pyplot as plt
import cv2 as cv
import inaccel.coral as inaccel
import numpy as np

### Read the two images

In [ ]:
left_img = cv.imread('data/left.png')
right_img = cv.imread('data/right.png')

In [ ]:
plt.figure(num = None, figsize = (10, 10), dpi = 80, facecolor = 'w', edgecolor = 'k')
plt.imshow(left_img)
plt.title('left')
plt.show()

plt.figure(num = None, figsize = (10, 10), dpi = 80, facecolor = 'w', edgecolor = 'k')
plt.imshow(right_img)
plt.title('right')
plt.show()

### Prepare the input/output ndarrays

In [ ]:
with inaccel.allocator:
    left_mat = np.array(left_img[:,:,0])
    right_mat = np.array(right_img[:,:,0])
    disp_mat = np.ndarray(left_img.shape[0:2], dtype = np.uint16)

    cameraMA_l = np.array([933.173, 0.0, 663.451, 0.0, 933.173, 377.015, 0.0, 0.0, 1.0], dtype = np.float32)
    cameraMA_r = np.array([933.467, 0.0, 678.297, 0.0, 933.467, 359.623, 0.0, 0.0, 1.0], dtype = np.float32)

    distC_l = np.array([-0.169398, 0.0227329, 0.0, 0.0, 0.0], dtype = np.float32)
    distC_r = np.array([-0.170581, 0.0249444, 0.0, 0.0, 0.0], dtype = np.float32)

    irA_l = np.array([0.0011976323, -0.0000000019, -0.8153011732, 0.0000000007, 0.0011976994, \
                      -0.4422348617,  0.0000126839,  0.0000001064, 0.9913820905], dtype=np.float32)
    irA_r = np.array([0.0011976994,  0.0000000000, -0.8047567905, -0.0000000000, 0.0011976994, \
                      -0.4420566166, -0.0000000000, -0.0000001064,  1.0000392898], dtype=np.float32)

    bm_state_arr = np.array([0, 15, 31, 15, 0, 48, 20, 15, 16, 3, 0], dtype = np.int32)

### Generate a new acceleration request for stereoBM

In [ ]:
req = inaccel.request('com.xilinx.vitis.vision.stereoBM')
req.arg(left_mat).arg(right_mat)
req.arg(disp_mat)
req.arg(cameraMA_l).arg(cameraMA_r)
req.arg(distC_l).arg(distC_r)
req.arg(irA_l).arg(irA_r)
req.arg(bm_state_arr)
req.arg(np.int32(left_img.shape[0]))
req.arg(np.int32(right_img.shape[1]))

### Submit that request to Coral and wait for it's completion 

In [ ]:
inaccel.submit(req).result()

In [ ]:
disp_img = (disp_mat.view(np.ndarray) * (256.0 / 48.0) / (16.0)).astype(np.uint8)

plt.figure(num = None, figsize = (10, 10), dpi = 80, facecolor = 'w', edgecolor = 'k')
plt.imshow(disp_img, cmap = 'gray', vmin = 0, vmax = 255)
plt.title('disparity')
plt.show()